In [220]:
import json

with open('minors_v2.json', 'r') as f:
    minor_data = json.load(f)

with open('gened_v2.json', 'r') as f:
    gened_data = json.load(f)

with open('major_v2.json', 'r') as f:
    major_data = json.load(f)

with open('liberal_arts_courses.json', 'r') as f:
    lib_data = json.load(f)

with open('ID.json', 'r') as f:
    ID_data = json.load(f)

In [221]:
major_names = list(major_data.keys())
minor_names = list(minor_data.keys())

['Data Science BS', 'Applied Chemistry BS', 'Architecture Technology BTECH', 'Architecture BARCH', 'Computer Engineering Technology', 'Health Communication', 'Technology Teacher Education', 'Nursing BS', 'Cybersecurity']


In [222]:
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=major_names,
    description='Select a Major:',
    disabled=False,
)

def on_change(change):
    global selected_major
    if change['type'] == 'change' and change['name'] == 'value':
        selected_major = change['new']
        print(f"You selected: {selected_major}")

dropdown.observe(on_change)

display(dropdown)

Dropdown(description='Select a Major:', options=('Data Science BS', 'Applied Chemistry BS', 'Architecture Tech…

You selected: Architecture Technology BTECH
You selected: Data Science BS


In [223]:
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=minor_names,
    description='Select a Minor:',
    disabled=False,
)

def on_change(change):
    global selected_minor
    if change['type'] == 'change' and change['name'] == 'value':
        selected_minor = change['new']
        print(f"You selected: {selected_minor}")

dropdown.observe(on_change)

display(dropdown)

Dropdown(description='Select a Minor:', options=('Arabic Language and Cultural Studies', 'Art History & Visual…

You selected: Art History & Visual Culture
You selected: Business
You selected: Computer Science


In [244]:
courses = []  # Initialize an empty list to store course codes
if selected_minor in minor_data.keys():  # Check if the selected minor exists in minor_data
    minor_info = minor_data[selected_minor]
    for section_name, section_list in minor_info.items(): # Iterate through the items of the minor_info dictionary
        # section list is usually a list
        for entry in section_list:
            # Check if the entry is a dictionary and has a 'Course' key
            if isinstance(entry, dict) and 'Course' in entry:
                courses.append(entry['Course'])
            # Check if the entry is a dictionary and contains nested course options (like 'Choose 1')
            elif isinstance(entry, dict):
                for key, value in entry.items():
                    if isinstance(value, list):
                        for item in value:
                            if isinstance(item, dict) and 'Course' in item:
                                courses.append(item['Course'])


print(courses) # Print the extracted courses to verify

['MAT 1630', 'CST 1201', 'MAT 2440', 'MAT 2540', 'BIO 2110', 'CST 1204', 'CST 2402', 'CET 1111', 'MAT 2070', 'MAT 2571', 'MAT 2572', 'MAT 2580', 'MAT 2630', 'MAT 2675', 'MAT 2680', 'MAT 3672', 'MAT 3772', 'MAT 4672', 'PHYS 3600', 'PHYS 3600ID']


^^ it checks if we found the right minor, then goes inside its “Requirements” and collects every course code.
if there’s a “choose 1” section, it digs into that too and adds those course codes to the list.

In [245]:
gened_areas = ['EC', 'MQR', 'LPS', 'WCGI', 'USED', 'IS', 'CE', 'SW' ,'Lib Art', 'Adv Lib Art', 'ID', 'Major_Core','Major_Elec']
course_gened_fulfillment_list = []

In [246]:
gened_fulfillment_map = {}
for core_type in ['Required Core', 'Flexible Core']:
    for course_info in gened_data['gened'][core_type]:
        areas = course_info.get('Area(s)', [])
        gened_fulfillment_map[course_info['Course']] = areas

In [247]:
#set of course in the core and electives for followup processing
major_core = set([course['Course'] for course in major_data[selected_major]['Core']])
major_elective = set([course['Course'] for course in major_data[selected_major]['Electives']])
Adv_lib = set([course['Course'] for course in lib_data['Adv Lib Art']])
Lib_art = set([course['Course'] for course in lib_data['Lib Art']])

In [248]:
import re

course_gened_fulfillment_list = [] # Clear the list before populating it

for course in courses:
    fulfilled_geneds = gened_fulfillment_map.get(course, [])

    if not isinstance(fulfilled_geneds, list):
            fulfilled_geneds = [fulfilled_geneds] if fulfilled_geneds else []

    # Check for major core and elective fulfillment
    if course in major_core:
        fulfilled_geneds.append('Major_Core')
    if course in major_elective:
        fulfilled_geneds.append('Major_Elec')

    # Check for Liberal Arts fulfillment
    if course in Lib_art:
        fulfilled_geneds.append('Lib Art')
    if course in Adv_lib:
        fulfilled_geneds.append('Lib Art')
        fulfilled_geneds.append('Adv Lib Art')

    # Check for ID fulfillment
    if course in ID_data or re.search(r'\d+ID$', course):
        fulfilled_geneds.append('ID')


    course_gened_fulfillment_list.append({
        'Course': course,
        'Fulfilled Areas': fulfilled_geneds
    })

In [249]:
import pandas as pd

course_fulfillment_list_for_df = []
gened_areas_for_columns = gened_areas

for course_info in course_gened_fulfillment_list:
    course_name = course_info['Course']
    fulfilled_areas = course_info['Fulfilled Areas']
    course_dict = {'Course': course_name}
    for gened_abbr in gened_areas_for_columns:
        course_dict[gened_abbr] = 'X' if gened_abbr in fulfilled_areas else ''
    course_fulfillment_list_for_df.append(course_dict)

course_fulfillment_df = pd.DataFrame(course_fulfillment_list_for_df)
course_fulfillment_df = course_fulfillment_df.fillna('')

# Display the DataFrame with dotted horizontal and vertical lines
display(course_fulfillment_df.style.set_properties(**{'border': '1px dotted black'}))

,Course,EC,MQR,LPS,WCGI,USED,IS,CE,SW,Lib Art,Adv Lib Art,ID,Major_Core,Major_Elec
0,MAT 1630,,,,,,,,,X,,,,
1,CST 1201,,,,,,,,,,,,X,
2,MAT 2440,,,,,,,,X,X,X,,X,
3,MAT 2540,,,,,,,,X,X,X,,,
4,BIO 2110,,,,,,,,,,,,,
5,CST 1204,,,,,,,,,,,,X,
6,CST 2402,,,,,,,,,,,,X,
7,CET 1111,,,,,,,,,,,,,
8,MAT 2070,,,,,,,,,,,,,
9,MAT 2571,,,,,,,,,,,,,
